# Imports

In [257]:
# Librairies
import numpy as np
import pandas as pd

In [258]:
# Fichiers
# url = 'sources/' # Jupyter (local)
url = 'https://raw.githubusercontent.com/gllmfrnr/oc/master/p4/sources/' # Github
customers =       pd.read_csv(url + 'customers.csv')
products =        pd.read_csv(url + 'products.csv')    
transactions =    pd.read_csv(url + 'transactions.csv')

In [259]:
# Styles
# Exemple : print(color.bold + 'Hello' + color.end)
class style:
   blue = '\033[94m'
   green = '\033[92m'
   red = '\033[91m'
   bold = '\033[1m'
   underline = '\033[4m'
   end = '\033[0m' 

In [260]:
# Fonction d'exploration des datasets
def exploration(df):
    print( 
        style.red + 'SAMPLE ↓ \n' + style.end,
        df.sample(3), '\n'                                              # Sample
        ) 
    print(style.red + 'INFO ↓ \n' + style.end)
    print(df.info(), '\n')                                              # Colonnes, null-count, types d'objets
    for i in range(0, len(df.columns)):
        print(
            style.red + 'COLONNE',
            style.bold + df.columns[i], ': \n' + style.end,             # Nom de colonne
             df.iloc[:, i].describe(), '\n',                            # .describe()
            sum(df.iloc[:, i].isna()), 'Nan \n',                        # Nbre de Nan 
            df.shape[0] - np.count_nonzero(df.iloc[:, i]), 'zéros \n',  # Nbre de 0
            len(df.iloc[:, i].unique()), 'valeurs uniques : \n',
            df.iloc[:, i].unique(), '\n'                                # Nombre de valeurs uniques 
        )

In [261]:
# Fonction de recherche de clé primaire (une seule colonne)
def pk(df,col): # Ne pas oublier les guillemets en déclarant le paramètre col
    x = 0
    df = df.reset_index()  
    for i in range(0,len(df)):   
        
        searched = df.loc[i][col] # Valeur à rechercher
    
        s = df[df[col]==searched].drop_duplicates() # Rechercher cette valeur dans toute la df, et supprimer les duplicates pour vérifier combien de lignes on obtient
  
        if len(s) == 1: # Si on obtient une seule ligne, c'est a priori une clé primaire
            x += 1
       
    if x == len(df): # Si pour chaque ligne de la df, la requêtes a ajouté 1 à x, alors x = la longueur de la dataframe
        print('\'', col, '\' est bien une clé primaire.')
    else:
        print('\'', col, '\' n\'est pas une clé primaire.')

# Fonction de recherche sur toute la dataframe
def dataframekeys(df):
  for i in range(0, len(df.columns)):
      print(pk(df, df.columns[i]))

# Exploration

## Datasets

### Customers
- 3 colonnes
    - client_id → identifiant unique pour chaque client (aucun doublon)
    - sex → genre (2 valeurs possibles : f ou m)
    - birth → année de naissance
- Clé primaire : **'client_id'**
- Aucune valeur dans le dataset n'est manquante
- Aucune valeur dans le dataset ne parait aberrante

In [262]:
# dataframekeys(customers)

# ' client_id ' est bien une clé primaire.
# ' sex ' n'est pas une clé primaire.
# ' birth ' n'est pas une clé primaire.

In [263]:
exploration(customers)

SAMPLE ↓ 
      client_id sex  birth
6895    c_2391   f   2002
2555    c_6367   m   1991
7657    c_7671   m   2004 

INFO ↓ 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8623 entries, 0 to 8622
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   client_id  8623 non-null   object
 1   sex        8623 non-null   object
 2   birth      8623 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 202.2+ KB
None 

COLONNE client_id : 
 count       8623
unique      8623
top       c_4085
freq           1
Name: client_id, dtype: object 
 0 Nan 
 0 zéros 
 8623 valeurs uniques : 
 ['c_4410' 'c_7839' 'c_1699' ... 'c_5119' 'c_5643' 'c_84'] 

COLONNE sex : 
 count     8623
unique       2
top          f
freq      4491
Name: sex, dtype: object 
 0 Nan 
 0 zéros 
 2 valeurs uniques : 
 ['f' 'm'] 

COLONNE birth : 
 count    8623.000000
mean     1978.280877
std        16.919535
min      1929.000000
25%      1966.000000
50%      1

### Products
- 3 colonnes
  - id_prod : identifiant produit unique
  - price : prix (1455 valeurs différentes, pour 3287 produits-lignes)
  - categ : catégorie (3 valeurs différentes : 0, 1, 2)
- Clé primaire : **id_prod**
- Aucune valeur n'est manquante dans le dataset
- La colonne prix semble contenir des valeurs aberrantes (inférieures à 0)

In [264]:
# dataframekeys(products)

# ' id_prod ' est bien une clé primaire.
# ' price ' n'est pas une clé primaire.
# ' categ ' n'est pas une clé primaire.

In [265]:
exploration(products)

SAMPLE ↓ 
      id_prod  price  categ
946   0_2169   0.99      0
305    1_250  20.76      1
1665  0_1757   1.99      0 

INFO ↓ 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3287 entries, 0 to 3286
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   id_prod  3287 non-null   object 
 1   price    3287 non-null   float64
 2   categ    3287 non-null   int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 77.2+ KB
None 

COLONNE id_prod : 
 count     3287
unique    3287
top       0_62
freq         1
Name: id_prod, dtype: object 
 0 Nan 
 0 zéros 
 3287 valeurs uniques : 
 ['0_1421' '0_1368' '0_731' ... '0_802' '1_140' '0_1920'] 

COLONNE price : 
 count    3287.000000
mean       21.856641
std        29.847908
min        -1.000000
25%         6.990000
50%        13.060000
75%        22.990000
max       300.000000
Name: price, dtype: float64 
 0 Nan 
 0 zéros 
 1455 valeurs uniques : 
 [19.99  5.13 17.99 ... 11.22 38.

### Transactions
*4 colonnes :*
- 2 clés étrangères
  - id_prod (vers **products**)
  - client_id (vers **customers**)
- date : date de la transaction (336855 valeurs uniques)
- session_id : identifiant de session (169195 valeurs uniques)

*Insights :*
- Une session peut contenir plusieurs dates
- Il y a sensiblement moins de produits et de clients que dans les tables étrangères que référencent id_prod et client_id
- On constate que la valeur top de 'date' semble aberrante


In [266]:
exploration(transactions)

SAMPLE ↓ 
        id_prod                        date session_id client_id
263126   1_683  2021-12-24 06:50:59.488479   s_139161    c_8615
91768    1_685  2022-02-15 00:00:45.946229   s_165274     c_950
250092  0_1489  2021-05-26 03:49:33.759431    s_39614    c_2232 

INFO ↓ 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 337016 entries, 0 to 337015
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   id_prod     337016 non-null  object
 1   date        337016 non-null  object
 2   session_id  337016 non-null  object
 3   client_id   337016 non-null  object
dtypes: object(4)
memory usage: 10.3+ MB
None 

COLONNE id_prod : 
 count     337016
unique      3266
top        1_369
freq        1081
Name: id_prod, dtype: object 
 0 Nan 
 0 zéros 
 3266 valeurs uniques : 
 ['0_1483' '2_226' '1_374' ... '0_833' '0_1284' '0_1116'] 

COLONNE date : 
 count                              337016
unique                             336

## Data (jointures)

In [267]:
# Jointure de customers sur transactions
data = pd.merge(transactions, customers, on="client_id")

# Jointure de products sur transactions
data = pd.merge(data, products, on="id_prod")

data

,id_prod,date,session_id,client_id,sex,birth,price,categ
0,0_1483,2021-04-10 18:37:28.723910,s_18746,c_4450,f,1977,4.99,0
1,0_1483,2021-12-27 11:11:12.123067,s_140787,c_5433,f,1981,4.99,0
2,0_1483,2021-10-27 04:56:38.293970,s_110736,c_857,m,1985,4.99,0
3,0_1483,2021-07-04 06:43:45.676567,s_57626,c_3679,f,1989,4.99,0
4,0_1483,2021-09-19 08:45:43.735331,s_92165,c_1609,m,1980,4.99,0
...,...,...,...,...,...,...,...,...
336908,0_1920,2021-04-13 18:36:10.252971,s_20115,c_7088,m,1987,25.16,0
336909,0_1920,2021-05-30 02:37:22.371278,s_41465,c_7748,f,1989,25.16,0
336910,2_23,2021-09-27 04:47:02.271354,s_96170,c_3976,f,1992,115.99,2
336911,2_28,2021-05-11 01:31:34.932056,s_32812,c_7613,f,1993,103.50,2


In [268]:
exploration(data)

SAMPLE ↓ 
        id_prod                        date session_id  ... birth  price  categ
255726  0_1421  2021-06-12 20:10:39.470832    s_47769  ...  1976  19.99      0
5084    0_1407  2021-08-03 05:15:29.691727    s_70704  ...  1985  13.99      0
203136   0_537  2021-09-04 00:59:46.304192    s_84729  ...  1976  16.99      0

[3 rows x 8 columns] 

INFO ↓ 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 336913 entries, 0 to 336912
Data columns (total 8 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   id_prod     336913 non-null  object 
 1   date        336913 non-null  object 
 2   session_id  336913 non-null  object 
 3   client_id   336913 non-null  object 
 4   sex         336913 non-null  object 
 5   birth       336913 non-null  int64  
 6   price       336913 non-null  float64
 7   categ       336913 non-null  int64  
dtypes: float64(1), int64(2), object(5)
memory usage: 23.1+ MB
None 

COLONNE id_prod : 
 count     336913
uni

## Valeurs aberrantes

### Colonne **price**
- contient 200 fois la valeur '-1' (le minimum) 
- contient 8 fois la valeur '300' (le maximum) : la colonne ne contient pourtant pas d'entiers (***à vérifier***), ni les valeurs '100' ou '200'

In [269]:
# 200 fois la valeur '-1'
data.sort_values(by=['price']).head(202)

,id_prod,date,session_id,client_id,sex,birth,price,categ
336029,T_0,test_2021-03-01 02:30:02.237426,s_0,ct_0,f,2001,-1.00,0
335975,T_0,test_2021-03-01 02:30:02.237426,s_0,ct_0,f,2001,-1.00,0
335974,T_0,test_2021-03-01 02:30:02.237442,s_0,ct_0,f,2001,-1.00,0
335973,T_0,test_2021-03-01 02:30:02.237443,s_0,ct_0,f,2001,-1.00,0
335972,T_0,test_2021-03-01 02:30:02.237441,s_0,ct_0,f,2001,-1.00,0
...,...,...,...,...,...,...,...,...
335878,T_0,test_2021-03-01 02:30:02.237420,s_0,ct_1,m,2001,-1.00,0
336043,T_0,test_2021-03-01 02:30:02.237417,s_0,ct_0,f,2001,-1.00,0
336044,T_0,test_2021-03-01 02:30:02.237416,s_0,ct_0,f,2001,-1.00,0
334673,0_202,2021-10-21 01:34:57.453011,s_107804,c_2276,f,1978,0.62,0


In [270]:
# 8 fois la valeur '300'
data.sort_values(by=['price'], ascending=False).head(10)

,id_prod,date,session_id,client_id,sex,birth,price,categ
332082,2_2,2021-03-07 14:07:31.004391,s_3041,c_1001,m,1982,300.00,2
332081,2_2,2022-02-28 21:24:28.442508,s_172373,c_659,f,2001,300.00,2
332078,2_2,2021-03-26 04:33:19.697459,s_11582,c_4958,m,1999,300.00,2
332079,2_2,2021-12-07 06:52:26.810576,s_130658,c_4958,m,1999,300.00,2
332080,2_2,2021-12-04 15:45:14.708313,s_129351,c_4958,m,1999,300.00,2
332085,2_2,2021-05-02 18:34:47.753888,s_28956,c_2329,f,1996,300.00,2
332084,2_2,2021-06-12 08:44:00.882917,s_47558,c_5237,m,1999,300.00,2
332083,2_2,2021-06-06 23:14:33.815188,s_45088,c_2467,f,1997,300.00,2
336890,2_76,2021-11-20 02:53:26.747301,s_122253,c_2221,m,1997,254.44,2
336891,2_76,2021-08-12 14:52:15.909705,s_74790,c_2791,m,2004,254.44,2


In [271]:
# 100 et 200 ne figurent pas dans la colonne
print(
    data[data['price']==100], '\n\n',
    data[data['price']==200], '\n\n',
    data[data['price']==300]
    )

Empty DataFrame
Columns: [id_prod, date, session_id, client_id, sex, birth, price, categ]
Index: [] 

 Empty DataFrame
Columns: [id_prod, date, session_id, client_id, sex, birth, price, categ]
Index: [] 

        id_prod                        date session_id  ... birth  price  categ
332078     2_2  2021-03-26 04:33:19.697459    s_11582  ...  1999  300.0      2
332079     2_2  2021-12-07 06:52:26.810576   s_130658  ...  1999  300.0      2
332080     2_2  2021-12-04 15:45:14.708313   s_129351  ...  1999  300.0      2
332081     2_2  2022-02-28 21:24:28.442508   s_172373  ...  2001  300.0      2
332082     2_2  2021-03-07 14:07:31.004391     s_3041  ...  1982  300.0      2
332083     2_2  2021-06-06 23:14:33.815188    s_45088  ...  1997  300.0      2
332084     2_2  2021-06-12 08:44:00.882917    s_47558  ...  1999  300.0      2
332085     2_2  2021-05-02 18:34:47.753888    s_28956  ...  1996  300.0      2

[8 rows x 8 columns]


### (essai : liste des entiers dans price)

In [272]:
# 5.00.is_integer()

#a = []

#for i in range(0, len(data)):
  #a.append(data['price'].loc[i].is_integer())

In [273]:
# pd.DataFrame([data['price'].loc[i].is_integer()], columns=['A']) for i in range(0, len(data)], ignore_index=True)

In [274]:


#dftest = pd.DataFrame(columns = ["a", "b"])


#for i in range(0, len(data)):


 # dftest = dftest.append({'a': data['price'].loc[i]}, ignore_index=True)
  #dftest = dftest.append({'b': data['price'].loc[i].is_integer()}, ignore_index=True)

### Colonne **date**
- Toutes les valeurs sont postérieures à la date d'extraction des données (2 avril 2020)
- 2 années seulement : 2021 et 2022

In [275]:
# Les 200 dernières valeurs sont contiennent le préfixe 'test_'
data.sort_values(by=['date'], ascending=False).head(202)

,id_prod,date,session_id,client_id,sex,birth,price,categ
335980,T_0,test_2021-03-01 02:30:02.237450,s_0,ct_0,f,2001,-1.00,0
336033,T_0,test_2021-03-01 02:30:02.237449,s_0,ct_0,f,2001,-1.00,0
335944,T_0,test_2021-03-01 02:30:02.237449,s_0,ct_1,m,2001,-1.00,0
336032,T_0,test_2021-03-01 02:30:02.237449,s_0,ct_0,f,2001,-1.00,0
336070,T_0,test_2021-03-01 02:30:02.237448,s_0,ct_0,f,2001,-1.00,0
...,...,...,...,...,...,...,...,...
335887,T_0,test_2021-03-01 02:30:02.237412,s_0,ct_1,m,2001,-1.00,0
335882,T_0,test_2021-03-01 02:30:02.237412,s_0,ct_1,m,2001,-1.00,0
335906,T_0,test_2021-03-01 02:30:02.237412,s_0,ct_1,m,2001,-1.00,0
276460,0_1775,2022-02-28 23:59:58.040472,s_172423,c_1460,m,1989,6.99,0


In [276]:
# Vérifier si la colonne ne contient effectivement que 200 x la chaîne 'test'
data['date'].str.contains('test').value_counts()

False    336713
True        200
Name: date, dtype: int64

# Nettoyage

## Suppression des 200 lignes-test
La valeur '200' revient plusieurs fois dans l'analyse des aberrations : 
- 200 fois le préfixe 'test_' dans la colonne date
- 200 fois la valeur '-1' dans la colonne price'

Elles figurent dans les mêmes lignes : on supprime ces données

In [277]:
# Montrer que la combinaison des 2 valeurs ne renvoie bien que 200 lignes
lignes_test = data[(data['price']==-1) & (data['date'].str.contains('test'))]

print(lignes_test)

       id_prod                             date session_id  ... birth price  categ
335878     T_0  test_2021-03-01 02:30:02.237420        s_0  ...  2001  -1.0      0
335879     T_0  test_2021-03-01 02:30:02.237446        s_0  ...  2001  -1.0      0
335880     T_0  test_2021-03-01 02:30:02.237414        s_0  ...  2001  -1.0      0
335881     T_0  test_2021-03-01 02:30:02.237434        s_0  ...  2001  -1.0      0
335882     T_0  test_2021-03-01 02:30:02.237412        s_0  ...  2001  -1.0      0
...        ...                              ...        ...  ...   ...   ...    ...
336073     T_0  test_2021-03-01 02:30:02.237437        s_0  ...  2001  -1.0      0
336074     T_0  test_2021-03-01 02:30:02.237438        s_0  ...  2001  -1.0      0
336075     T_0  test_2021-03-01 02:30:02.237436        s_0  ...  2001  -1.0      0
336076     T_0  test_2021-03-01 02:30:02.237445        s_0  ...  2001  -1.0      0
336077     T_0  test_2021-03-01 02:30:02.237430        s_0  ...  2001  -1.0      0

[20

In [278]:
# Nombre de lignes avant nettoyage
print('Nombre de lignes avant nettoyage :', len(data))

# Supprimer les lignes test de la dataframe, via leur index
data.drop(lignes_test.index, inplace=True)

# Nombre de lignes après nettoyage
print('Nombre de lignes après nettoyage :', len(data))

Nombre de lignes avant nettoyage : 336913
Nombre de lignes après nettoyage : 336713
